In [42]:
# !pip install spacy==3.0.0
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import random
from tqdm import tqdm
from spacy.matcher import Matcher
import datetime as dt
from spacy.tokens import Doc
from spacy.util import minibatch, compounding
# from spacy.training import Example

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [ ]:
df = pd.read_csv('/kaggle/input/scl-2021-ds/train.csv')

In [ ]:
df['POI'] = df['POI/street'].str.split("/", n = 2, expand = True)[0]
df['street'] = df['POI/street'].str.split("/", n = 2, expand = True)[1]

In [ ]:
nlp = spacy.blank("id")

In [ ]:
doc_list_raw_address = []
doc_list_poi = []
doc_list_street = []
for _, row in df.iterrows():
    doc = nlp(row.raw_address)
    doc_list_raw_address.append(doc)
    doc = nlp(row.POI)
    doc_list_poi.append(doc)
    doc = nlp(row.street)
    doc_list_street.append(doc)

In [ ]:
train_data = []
for i in tqdm(range(300000)):
    if doc_list_poi[i].text:
        a = [(d.idx, d.idx + len(d.text)) for d in doc_list_raw_address[i] if d.text == doc_list_poi[i][0].text]
        if (a):
            b = [(d.idx, d.idx + len(d.text), 'POI') for d in doc_list_raw_address[i] if d.text == doc_list_poi[i][-1].text]
            if (b):
                train_data.append((doc_list_raw_address[i], {'entities' : [(a[0][0], b[0][1], 'POI')]}))
                
for i in tqdm(range(300000)):
    if doc_list_street[i].text:
        a = [(d.idx, d.idx + len(d.text)) for d in doc_list_raw_address[i] if d.text == doc_list_street[i][0].text]
        if (a):
            b = [(d.idx, d.idx + len(d.text)) for d in doc_list_raw_address[i] if d.text == doc_list_street[i][-1].text]
            if (b):
                train_data.append((doc_list_raw_address[i], {'entities' : [(a[0][0], b[0][1], 'street')]}))

In [ ]:
for _, row in df.replace("/", '').iterrows():
    if row['POI/street']:
        pass
    else:
        train_data.append((row['raw_address'], {'entities' : []}))

In [ ]:
len(train_data)

In [ ]:
def create_blank_nlp(train_data):
    nlp = spacy.blank("id")
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
    ner = nlp.get_pipe("ner")
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    return nlp

In [ ]:
nlp = create_blank_nlp(train_data)

In [43]:
optimizer = nlp.begin_training()
for i in range(50):
    random.shuffle(train_data)
    losses = {}
    batches = minibatch(train_data[:100000], size=compounding(50, 1000, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
            texts,
            annotations,
            drop=0.1,
            losses=losses,
        )
    print(f"Losses at iteration {i} - {dt.datetime.now()} {losses}")

1103it [02:59,  6.16it/s]


Losses at iteration 0 - 2021-03-15 08:12:09.351731 {'ner': 65124.469610814005}


1103it [02:57,  6.22it/s]


Losses at iteration 1 - 2021-03-15 08:15:07.272049 {'ner': 64672.536898996215}


1103it [02:57,  6.22it/s]


Losses at iteration 2 - 2021-03-15 08:18:05.135572 {'ner': 64601.411274164915}


1103it [02:59,  6.16it/s]


Losses at iteration 3 - 2021-03-15 08:21:04.656241 {'ner': 64254.68258206546}


63it [00:06,  9.58it/s]


KeyboardInterrupt: 

In [45]:
testdf = pd.read_csv("../input/scl-2021-ds/test.csv")

In [46]:
nlp.to_disk('nlp_model')
nlp_model = spacy.load('nlp_model')

In [47]:
testdf.head()

,id,raw_address
0,0,s. par 53 sidanegara 4 cilacap tengah
1,1,"angg per, baloi indah kel. lubuk baja"
2,2,"asma laun, mand imog,"
3,3,"ud agung rej, raya nga sri wedari karanganyar"
4,4,"cut mutia, 35 baiturrahman"


In [50]:
out = []

for _, row in tqdm(testdf.iterrows()):
    doc = nlp_model(row.raw_address)
    text = ''
    for ent in doc.ents:
        if ent.label_ == 'POI':
            text += ent.text
            text += '/'
        else:
            text += '/'
            text += ent.text
    if text:
        out.append(text.replace("//", "/"))
    else:
        out.append('/')

50000it [03:01, 276.01it/s]


In [51]:
submission = pd.DataFrame({'id':testdf['id'], 'POI/street': out})

In [52]:
submission.head()

,id,POI/street
0,0,/s. par
1,1,/angg per
2,2,/mand imog
3,3,/raya nga sri
4,4,/cut mutia


In [63]:
submission.to_csv('./7PERCENT.csv', header=True, index=False)